# Spotify song recommender

### 1- import everything you need :D

In [43]:
import os
import json
import requests
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

True

### 2- define a function that gets you access to the spotify API

In [44]:
def spotify_token ():
    
    """This function refreshes a 'queries token' for a given app on Spotify.
    queries token are refreshed every 3600 seconds
    returns: token as a string
    """

    # 1. Defining: credentials fot the app
    
    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    
    #2. Request
    
    body_params = {"grant_type":"client_credentials"}
    url = "https://accounts.spotify.com/api/token"
    response = requests.post(url, data=body_params, auth=(CLIENT_ID,CLIENT_SECRET))
    
    try:
        token = response.json()["access_token"]
        return token

    except:
        print("Nope. Maybe check credentials?")


In [45]:
token = spotify_token()
token

'BQAAn7eZGVymjHeFpH50REMRoz4tUB-aEq3IuERLggmkTcYFLCbI66MXx76tAPdxarv9xd59HlKLTCHFIDOQZU67TtWL3FhKwAJHgMjHMOzYvr0bAoQ'

### 3- define function for the user to input the link of the track 

In [46]:
def extract_id_from_link(link):
    #link = input("Enter the link: ")
    # Extract the ID between the last '/' and the '?'
    start_index = link.rfind("/") + 1
    end_index = link.find("?")
    
    # Return the extracted ID
    if end_index != -1:
        return link[start_index:end_index]
    else:
        return link[start_index:]

# Call the function to extract the ID from the user-provided link
#song_id = extract_id_from_link("https://open.spotify.com/track/1467Zd8OEpN4uMLVrplVlr?si=178e274dc1524fc1")
#song_id


### 4- define a function to request spotify

In [47]:
def request_from_spotify (song_id):
    headers = {"Authorization":f"Bearer {token}"}
    url_base = "https://api.spotify.com/v1/recommendations?seed_tracks="
    url = f"https://api.spotify.com/v1/recommendations?limit=5&seed_tracks={song_id}"
    return requests.get(url, headers=headers)

### 5- define a function to get response from spotify

In [48]:
def todo (link):
    #link = input("Enter the link: ")
    song_id = extract_id_from_link(link)
    res = request_from_spotify (song_id)
    return res

In [49]:
res = todo("https://open.spotify.com/track/1467Zd8OEpN4uMLVrplVlr?si=178e274dc1524fc1") # https://open.spotify.com/track/1467Zd8OEpN4uMLVrplVlr?si=178e274dc1524fc1
res

<Response [200]>

In [50]:
#res.json()["tracks"]

In [51]:
album_name = res.json()["tracks"][0]["album"]["name"]
album_name

'Endless Sacrifice'

In [52]:
song_name = res.json()["tracks"][0]["name"]
song_name

'Harmed'

In [53]:
artists = res.json()["tracks"][0]["artists"][0]["name"]
artists

'Fideles'

### 6- define a function to put everything on a dictionary

In [54]:
def songs_to_dict(link):
    res = todo(link)
    song_name = res.json()["tracks"][0]["name"]
    artists = res.json()["tracks"][0]["artists"][0]["name"]
    album_name = res.json()["tracks"][0]["album"]["name"]
    
    song_dict = {
    "song": song_name,
    "artist": artists,
    "album": album_name
    }
    return song_dict

In [55]:
songs_to_dict("https://open.spotify.com/track/1467Zd8OEpN4uMLVrplVlr?si=178e274dc1524fc1")

{'song': 'Type Machine - Hexlogic Remix',
 'artist': '8Kays',
 'album': 'Type Machine'}

In [56]:
res.json()["tracks"][0]["external_urls"]["spotify"]

'https://open.spotify.com/track/2UCDEXstmpgvf1akDnYRSh'

### 7- define the final function that will have everything together

In [57]:
#def song_recommendation(res):
    #new_list = []
    #for i in res.json()["tracks"]: #i -> link
        #url = i["external_urls"]["spotify"]
        #new_list.append(songs_to_dict(url))
    #return pd.DataFrame(new_list)


In [58]:
def song_recommendation(res):
    new_list = []
    
    link = input("Enter the link: ")
    song_id = extract_id_from_link(link)
    res = request_from_spotify (song_id)
    
    res = todo(link)
    song_name = res.json()["tracks"][0]["name"]
    artists = res.json()["tracks"][0]["artists"][0]["name"]
    album_name = res.json()["tracks"][0]["album"]["name"]
    
    song_dict = {
    "song": song_name,
    "artist": artists,
    "album": album_name
    }    
    
    for i in res.json()["tracks"]: #i -> link
        url = i["external_urls"]["spotify"]
        new_list.append(songs_to_dict(url))
    return pd.DataFrame(new_list)

In [59]:
song_recommendation(res)

Enter the link: https://open.spotify.com/track/04mq28ok538qTCtlcApfs9?si=dd5333f89c1c4ff0


,song,artist,album
0,Confusión,Poligamia,Vueltas & Vueltas
1,Princesa,Alerta Kamarada,Somos Uno
2,Mi Mantra,Sie7e,Gaia
3,Cafecito,La Ciencia de Juancho Valencia,La Fórmula del Mambo
4,Canción Pequeña,Lucio Feuillet,Bailando Bailando


In [19]:
# 1. link_cancion -> id
# 2. id -> res
# 3. res -> df
# 4. function (link): 1(link), 2(id), 3(res) -> df